In [1]:
import glass
from ann_dataset import dataset_dict

In [ ]:
topk = 10
dataset = dataset_dict['sift-128-euclidean']('../datasets')
X_train = dataset.get_database()
X_test = dataset.get_queries()
Y = dataset.get_groundtruth(topk)
n, d = X_train.shape
nq, d = X_test.shape
metric = dataset.metric
print(f"n = {n}, d = {d}, nq = {nq}, metric = {metric}")
print(f"dataset size = {n * d * 4 / 1024 / 1024:.2f}MB")


In [ ]:
from time import time

index = glass.Index(index_type="HNSW", metric=metric, R=48, L=100, quant="SQ4U")
t0 = time()
g = index.build(X_train)
t1 = time()
print(f"Done build graph, elapsed {t1 - t0:.2f}s")


In [ ]:
s = glass.Searcher(graph=g, data=X_train, metric=metric, quantizer="SQ4U")
s.set_ef(36)


In [ ]:
from time import time

pred = s.batch_search(query=X_test, k=topk).reshape(-1, topk)
recall = dataset.evaluate(pred, topk)
print(f"Recall = {recall * 100:.2f}%")


In [ ]:
s.batch_search(query=X_test, k=topk)  # warmup
for iter in range(10):
    t1 = time()
    pred = s.batch_search(query=X_test, k=topk)
    t2 = time()
    print(f"QPS = {nq / (t2 - t1):.2f}")


In [ ]:
s.optimize()


In [ ]:
s.batch_search(query=X_test, k=topk)  # warmup
for iter in range(10):
    t1 = time()
    pred = s.batch_search(query=X_test, k=topk).reshape(-1, topk)
    t2 = time()
    print(f"QPS = {nq / (t2 - t1)}")
